<a href="https://colab.research.google.com/github/Linux-Server/AI_Engineering/blob/main/01_Choose_Right_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### [Choose the Right Model + Method]("https://docs.unsloth.ai/get-started/fine-tuning-llms-guide")
 - LoRA: Fine-tunes small, trainable matrices in 16-bit without updating all model weights.  
 - QLoRA: Combines LoRA with 4-bit quantization to handle very large models with minimal resources.



In [2]:
%%capture
!pip install unsloth

In [3]:
from unsloth import FastLanguageModel

load_in_4bit = True ## use 4 bit quantization to reduce memory
max_seq_length = 2048 ## choose any , unsloth support auto RoPE scalling internally
model_name = "unsloth/Phi-4"

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name=model_name,
    max_seq_length=max_seq_length,
    load_in_4bit=load_in_4bit
    )

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.9.11: Fast Llama patching. Transformers: 4.56.1.
   \\   /|    NVIDIA L4. Num GPUs = 1. Max memory: 22.161 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.8.0+cu126. CUDA: 8.9. CUDA Toolkit: 12.6. Triton: 3.4.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.32.post2. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors.index.json: 0.00B [00:00, ?B/s]

Fetching 3 files:   0%|          | 0/3 [00:00<?, ?it/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/1.03G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/4.39G [00:00<?, ?B/s]

...

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

 - Phi-4, Microsoft's new 14B model that performs on par with OpenAI's GPT-4o-mini

In [5]:
prompt  = [{"role": "user", "content": "Who are you"}]
text = "Who are you?"

raw_model_inputs = tokenizer.apply_chat_template(prompt,tokenize=False, add_generation_prompt=True)
raw_model_inputs

'<|im_start|>user<|im_sep|>Who are you<|im_end|><|im_start|>assistant<|im_sep|>'

In [7]:
model_inputs = tokenizer(raw_model_inputs, return_tensors="pt").to(model.device)
model_inputs

{'input_ids': tensor([[100264,    882, 100266,  15546,    527,    499, 100265, 100264,  78191,
         100266]], device='cuda:0'), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1]], device='cuda:0')}

In [10]:
outputs = model.generate(**model_inputs, max_new_tokens=100)
outputs



["userWho are youassistantI'm Phi, a language model developed by Microsoft. My purpose is to assist users by providing information, answering questions, and helping with a wide range of topics to the best of my ability. If you have any questions or need assistance, feel free to ask!"]

In [11]:
from pprint import pprint

pprint(tokenizer.batch_decode(outputs, skip_special_tokens=True))

["userWho are youassistantI'm Phi, a language model developed by Microsoft. My "
 'purpose is to assist users by providing information, answering questions, '
 'and helping with a wide range of topics to the best of my ability. If you '
 'have any questions or need assistance, feel free to ask!']


 - Now lets add LoRA adapters for PEFT

In [12]:
model = FastLanguageModel.get_peft_model(
    model=model,
    r=16,  # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    lora_alpha=16,  # Choose any number > 0 ! Suggested 16, 32, 64, 128
    lora_dropout=0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

Unsloth 2025.9.11 patched 40 layers with 40 QKV layers, 40 O layers and 40 MLP layers.


In [14]:
from datasets import load_dataset

dataset = load_dataset("mlabonne/FineTome-100k", split="train")

dataset

README.md:   0%|          | 0.00/982 [00:00<?, ?B/s]

data/train-00000-of-00001.parquet:   0%|          | 0.00/117M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/100000 [00:00<?, ? examples/s]

Dataset({
    features: ['conversations', 'source', 'score'],
    num_rows: 100000
})

In [15]:
dataset[0]

{'conversations': [{'from': 'human',
   'value': 'Explain what boolean operators are, what they do, and provide examples of how they can be used in programming. Additionally, describe the concept of operator precedence and provide examples of how it affects the evaluation of boolean expressions. Discuss the difference between short-circuit evaluation and normal evaluation in boolean expressions and demonstrate their usage in code. \n\nFurthermore, add the requirement that the code must be written in a language that does not support short-circuit evaluation natively, forcing the test taker to implement their own logic for short-circuit evaluation.\n\nFinally, delve into the concept of truthiness and falsiness in programming languages, explaining how it affects the evaluation of boolean expressions. Add the constraint that the test taker must write code that handles cases where truthiness and falsiness are implemented differently across different programming languages.'},
  {'from': 'gpt

In [16]:
from unsloth.chat_templates import standardize_sharegpt

dataset = standardize_sharegpt(dataset)
dataset[0]

Unsloth: Standardizing formats (num_proc=12):   0%|          | 0/100000 [00:00<?, ? examples/s]

{'conversations': [{'content': 'Explain what boolean operators are, what they do, and provide examples of how they can be used in programming. Additionally, describe the concept of operator precedence and provide examples of how it affects the evaluation of boolean expressions. Discuss the difference between short-circuit evaluation and normal evaluation in boolean expressions and demonstrate their usage in code. \n\nFurthermore, add the requirement that the code must be written in a language that does not support short-circuit evaluation natively, forcing the test taker to implement their own logic for short-circuit evaluation.\n\nFinally, delve into the concept of truthiness and falsiness in programming languages, explaining how it affects the evaluation of boolean expressions. Add the constraint that the test taker must write code that handles cases where truthiness and falsiness are implemented differently across different programming languages.',
   'role': 'user'},
  {'content': 

In [17]:
def formatting_prompts_func(examples):
    convos = examples["conversations"]
    texts = [
        tokenizer.apply_chat_template(
            convo, tokenize = False, add_generation_prompt = False
        )
        for convo in convos
    ]
    return { "text" : texts, }


In [18]:
dataset = dataset.map(
    formatting_prompts_func,
    batched=True,
)

Map:   0%|          | 0/100000 [00:00<?, ? examples/s]

In [20]:
pprint(dataset[0])

{'conversations': [{'content': 'Explain what boolean operators are, what they '
                               'do, and provide examples of how they can be '
                               'used in programming. Additionally, describe '
                               'the concept of operator precedence and provide '
                               'examples of how it affects the evaluation of '
                               'boolean expressions. Discuss the difference '
                               'between short-circuit evaluation and normal '
                               'evaluation in boolean expressions and '
                               'demonstrate their usage in code. \n'
                               '\n'
                               'Furthermore, add the requirement that the code '
                               'must be written in a language that does not '
                               'support short-circuit evaluation natively, '
                               'for

In [21]:
dataset[5]["conversations"]

[{'content': 'How do astronomers determine the original wavelength of light emitted by a celestial body at rest, which is necessary for measuring its speed using the Doppler effect?',
  'role': 'user'},
 {'content': 'Astronomers make use of the unique spectral fingerprints of elements found in stars. These elements emit and absorb light at specific, known wavelengths, forming an absorption spectrum. By analyzing the light received from distant stars and comparing it to the laboratory-measured spectra of these elements, astronomers can identify the shifts in these wavelengths due to the Doppler effect. The observed shift tells them the extent to which the light has been redshifted or blueshifted, thereby allowing them to calculate the speed of the star along the line of sight relative to Earth.',
  'role': 'assistant'}]

In [22]:
dataset[5]["text"]

'<|im_start|>user<|im_sep|>How do astronomers determine the original wavelength of light emitted by a celestial body at rest, which is necessary for measuring its speed using the Doppler effect?<|im_end|><|im_start|>assistant<|im_sep|>Astronomers make use of the unique spectral fingerprints of elements found in stars. These elements emit and absorb light at specific, known wavelengths, forming an absorption spectrum. By analyzing the light received from distant stars and comparing it to the laboratory-measured spectra of these elements, astronomers can identify the shifts in these wavelengths due to the Doppler effect. The observed shift tells them the extent to which the light has been redshifted or blueshifted, thereby allowing them to calculate the speed of the star along the line of sight relative to Earth.<|im_end|>'

In [26]:
from trl import SFTConfig, SFTTrainer
from transformers import DataCollatorForSeq2Seq
trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    data_collator = DataCollatorForSeq2Seq(tokenizer = tokenizer),
    packing = False, # Can make training 5x faster for short sequences.
    args = SFTConfig(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        num_train_epochs = 1, # Set this for 1 full training run.
        #max_steps = 30,
        learning_rate = 2e-4,
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
        report_to = "none", # Use this for WandB etc
    ),
)

Unsloth: Tokenizing ["text"] (num_proc=16):   0%|          | 0/100000 [00:00<?, ? examples/s]

In [27]:
from unsloth.chat_templates import train_on_responses_only

trainer = train_on_responses_only(
    trainer,
    instruction_part="<|im_start|>user<|im_sep|>",
    response_part="<|im_start|>assistant<|im_sep|>",
)

Map (num_proc=12):   0%|          | 0/100000 [00:00<?, ? examples/s]

In [28]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 100,000 | Num Epochs = 1 | Total steps = 12,500
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 4 x 1) = 8
 "-____-"     Trainable parameters = 65,536,000 of 14,725,043,200 (0.45% trained)


Step,Training Loss
1,0.523600
2,0.537900
3,0.785500
4,0.591800
5,0.560200
6,0.662500
7,0.404100
8,0.752300
9,0.565100
10,0.490700


KeyboardInterrupt: 